In [6]:
# Initial Dependencies

import pandas as pd
from ast import literal_eval 

from config import dialect, username, password, host, port, database

from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [7]:
#  DB Connection

connection_string = f"{dialect}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [8]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [9]:
# Identify list of tables in DB

Base.classes.keys()

['user_data_stg', 'user_data']

In [10]:
# Save reference to the tables
stg_user_data = Base.classes.user_data_stg
user_data_tb = Base.classes.user_data

In [11]:
# Clear Staging table before every new load
session = Session(engine)
session.query(stg_user_data).delete()
session.commit()

In [12]:
# File path to load the data
user_file_path = "../Resources/sample_us_users.csv"
base_user_data_df = pd.read_csv(user_file_path)
base_user_data_df.head()

,id,address,inserted_at
0,278ab326-2b6a-4b4d-aa9b-64caaf074bb3,"{'city': 'Bloomfield', 'state': 'US-IN', 'coun...",2019-05-07 17:30:27.718
1,ebf05c95-4234-4251-889b-7fe71cf01d77,"{'city': 'Houston', 'state': 'Texas', 'country...",2020-04-07 21:35:38.243
2,be19fd93-d135-497a-9870-b626c4b89353,"{'city': 'Annapolis', 'state': 'Md', 'country'...",2019-05-07 17:39:06.656
3,54a5ed13-08bd-4bdc-b929-d6a5603dc72b,"{'city': 'Fultondale ', 'state': 'Alabama ', '...",2019-05-07 17:33:20.592
4,5272addc-b7ce-449c-89c6-2ad868d4a9ff,"{'city': 'Jacksonville', 'state': 'NC', 'count...",2019-05-07 17:32:20.498


In [13]:
# Load data into staging table
base_user_data_df.to_sql('user_data_stg',connection, if_exists='append', index=False)
session.commit()

In [14]:
# In reflection, the table will be visible only when it has primary key.. So I created a temporary key and dropping it here
stg_base_user_data_df = pd.read_sql_table('user_data_stg', connection)
stg_base_user_data_df = stg_base_user_data_df.drop(columns = ['index_col'])
stg_base_user_data_df.head()

,id,address,inserted_at
0,278ab326-2b6a-4b4d-aa9b-64caaf074bb3,"{'city': 'Bloomfield', 'state': 'US-IN', 'coun...",2019-05-07 17:30:27.718
1,ebf05c95-4234-4251-889b-7fe71cf01d77,"{'city': 'Houston', 'state': 'Texas', 'country...",2020-04-07 21:35:38.243
2,be19fd93-d135-497a-9870-b626c4b89353,"{'city': 'Annapolis', 'state': 'Md', 'country'...",2019-05-07 17:39:06.656
3,54a5ed13-08bd-4bdc-b929-d6a5603dc72b,"{'city': 'Fultondale ', 'state': 'Alabama ', '...",2019-05-07 17:33:20.592
4,5272addc-b7ce-449c-89c6-2ad868d4a9ff,"{'city': 'Jacksonville', 'state': 'NC', 'count...",2019-05-07 17:32:20.498


In [16]:
print(stg_base_user_data_df.columns)
stg_base_user_data_df.iloc[:,1:2]

Index(['id', 'address', 'inserted_at'], dtype='object')


,address
0,"{'city': 'Bloomfield', 'state': 'US-IN', 'coun..."
1,"{'city': 'Houston', 'state': 'Texas', 'country..."
2,"{'city': 'Annapolis', 'state': 'Md', 'country'..."
3,"{'city': 'Fultondale ', 'state': 'Alabama ', '..."
4,"{'city': 'Jacksonville', 'state': 'NC', 'count..."
...,...
4995,"{'city': 'Washington ', 'state': 'US-CA', 'cou..."
4996,"{'city': 'West Memphis ', 'state': 'AR', 'coun..."
4997,"{'city': 'Waskom ', 'state': 'Texas', 'country..."
4998,"{'city': 'BARRINGTON', 'state': 'US-NH', 'coun..."


In [17]:
stg_base_user_data_df.iloc[:,1:2]=stg_base_user_data_df.iloc[:,1:2].applymap(literal_eval)

In [18]:
formatted_address = pd.json_normalize(stg_base_user_data_df.address)
formatted_address.columns = [f'address_{col}' for col in formatted_address.columns]

formatted_address

,address_city,address_state,address_country,address_postCode
0,Bloomfield,US-IN,US,47424
1,Houston,Texas,US,77084
2,Annapolis,Md,US,21403
3,Fultondale,Alabama,US,35068
4,Jacksonville,NC,US,28546
...,...,...,...,...
4995,Washington,US-CA,US,98589
4996,West Memphis,AR,US,72301
4997,Waskom,Texas,US,75692
4998,BARRINGTON,US-NH,US,03825


In [19]:
user_data_normalized_df = pd.concat([stg_base_user_data_df, formatted_address], axis = 1)
user_data_normalized_df

,id,address,inserted_at,address_city,address_state,address_country,address_postCode
0,278ab326-2b6a-4b4d-aa9b-64caaf074bb3,"{'city': 'Bloomfield', 'state': 'US-IN', 'coun...",2019-05-07 17:30:27.718,Bloomfield,US-IN,US,47424
1,ebf05c95-4234-4251-889b-7fe71cf01d77,"{'city': 'Houston', 'state': 'Texas', 'country...",2020-04-07 21:35:38.243,Houston,Texas,US,77084
2,be19fd93-d135-497a-9870-b626c4b89353,"{'city': 'Annapolis', 'state': 'Md', 'country'...",2019-05-07 17:39:06.656,Annapolis,Md,US,21403
3,54a5ed13-08bd-4bdc-b929-d6a5603dc72b,"{'city': 'Fultondale ', 'state': 'Alabama ', '...",2019-05-07 17:33:20.592,Fultondale,Alabama,US,35068
4,5272addc-b7ce-449c-89c6-2ad868d4a9ff,"{'city': 'Jacksonville', 'state': 'NC', 'count...",2019-05-07 17:32:20.498,Jacksonville,NC,US,28546
...,...,...,...,...,...,...,...
4995,2f57b4fe-209e-4a50-9b88-dbd781426257,"{'city': 'Washington ', 'state': 'US-CA', 'cou...",2020-07-27 21:23:49.308,Washington,US-CA,US,98589
4996,a77f31f3-4366-4f84-a69c-783eefc898e2,"{'city': 'West Memphis ', 'state': 'AR', 'coun...",2020-11-24 20:07:56.751,West Memphis,AR,US,72301
4997,bced2d09-494c-4c69-b705-510cc31efb9a,"{'city': 'Waskom ', 'state': 'Texas', 'country...",2020-07-10 09:50:43.564,Waskom,Texas,US,75692
4998,88a96628-4a1a-4c7b-b58f-98debe08641d,"{'city': 'BARRINGTON', 'state': 'US-NH', 'coun...",2020-11-28 17:16:33.072,BARRINGTON,US-NH,US,03825


In [20]:
formatted_user_data_df = user_data_normalized_df.drop(columns = ['address'])
formatted_user_data_df.rename(columns = {'address_postCode':'address_postcode'}, inplace = True)

In [21]:
formatted_user_data_df.nunique()

id                  5000
inserted_at         5000
address_city        2948
address_state        503
address_country        1
address_postcode    3897
dtype: int64

In [17]:
pd.options.display.max_rows = 100

In [22]:
nan_rows  = formatted_user_data_df[formatted_user_data_df.isna().any(axis=1)]
nan_rows

,id,inserted_at,address_city,address_state,address_country,address_postcode
45,951e69c8-f6ff-433c-8703-b860ae5652dd,2019-11-03 03:27:16.779,Louisville,None,US,36048
261,444db88c-3cc2-4fb0-9513-c8652a9d48f7,2019-10-31 12:49:24.336,gilroy,None,US,95020
341,be7d5002-420c-4cb4-9808-ecd0834f5750,2019-10-29 23:47:40.592,Modesto,None,US,95354
358,f8f18915-80be-45c1-a3aa-6264e381866e,2020-01-02 22:50:09.136,Framingham,None,US,01702
624,151466b8-2141-4576-a1a0-97a8a0bc2950,2019-11-03 03:56:49.660,parkville,None,US,21234
...,...,...,...,...,...,...
4636,54077387-9764-480e-8584-e40ea8cf1549,2019-11-03 23:26:35.464,Baltimore,None,US,21221
4850,4f27830c-7321-4bcf-ac39-5c1fd5a91aed,2019-10-30 22:28:18.650,DC,None,US,20010
4952,f400fbbd-1d8a-4856-94d9-3dbbd96fa9bb,2019-12-03 05:12:19.451,Prudenville,None,US,48651
4979,56af688f-ecee-4c6d-ab3f-7133a5e50b4f,2019-11-06 19:06:00.032,Fort worth,None,US,76028


In [23]:
formatted_user_data_df.fillna({'address_state':'US'}, inplace=True)
formatted_user_data_df

,id,inserted_at,address_city,address_state,address_country,address_postcode
0,278ab326-2b6a-4b4d-aa9b-64caaf074bb3,2019-05-07 17:30:27.718,Bloomfield,US-IN,US,47424
1,ebf05c95-4234-4251-889b-7fe71cf01d77,2020-04-07 21:35:38.243,Houston,Texas,US,77084
2,be19fd93-d135-497a-9870-b626c4b89353,2019-05-07 17:39:06.656,Annapolis,Md,US,21403
3,54a5ed13-08bd-4bdc-b929-d6a5603dc72b,2019-05-07 17:33:20.592,Fultondale,Alabama,US,35068
4,5272addc-b7ce-449c-89c6-2ad868d4a9ff,2019-05-07 17:32:20.498,Jacksonville,NC,US,28546
...,...,...,...,...,...,...
4995,2f57b4fe-209e-4a50-9b88-dbd781426257,2020-07-27 21:23:49.308,Washington,US-CA,US,98589
4996,a77f31f3-4366-4f84-a69c-783eefc898e2,2020-11-24 20:07:56.751,West Memphis,AR,US,72301
4997,bced2d09-494c-4c69-b705-510cc31efb9a,2020-07-10 09:50:43.564,Waskom,Texas,US,75692
4998,88a96628-4a1a-4c7b-b58f-98debe08641d,2020-11-28 17:16:33.072,BARRINGTON,US-NH,US,03825


In [24]:
cleansed_user_data_df = formatted_user_data_df[formatted_user_data_df.isna().any(axis=1)]
cleansed_user_data_df

,id,inserted_at,address_city,address_state,address_country,address_postcode


In [25]:
formatted_user_data_df.to_sql('user_data',connection, if_exists='append', index=False)
session.commit()

In [26]:
db_user_data_df = pd.read_sql_table('user_data', connection)
db_user_data_df.drop(columns = ['index_col'], inplace = True)
db_user_data_df.set_index('id')

,inserted_at,address_city,address_state,address_country,address_postcode
id,,,,,
278ab326-2b6a-4b4d-aa9b-64caaf074bb3,2019-05-07 17:30:27.718,Bloomfield,US-IN,US,47424
ebf05c95-4234-4251-889b-7fe71cf01d77,2020-04-07 21:35:38.243,Houston,Texas,US,77084
be19fd93-d135-497a-9870-b626c4b89353,2019-05-07 17:39:06.656,Annapolis,Md,US,21403
54a5ed13-08bd-4bdc-b929-d6a5603dc72b,2019-05-07 17:33:20.592,Fultondale,Alabama,US,35068
5272addc-b7ce-449c-89c6-2ad868d4a9ff,2019-05-07 17:32:20.498,Jacksonville,NC,US,28546
...,...,...,...,...,...
2f57b4fe-209e-4a50-9b88-dbd781426257,2020-07-27 21:23:49.308,Washington,US-CA,US,98589
a77f31f3-4366-4f84-a69c-783eefc898e2,2020-11-24 20:07:56.751,West Memphis,AR,US,72301
bced2d09-494c-4c69-b705-510cc31efb9a,2020-07-10 09:50:43.564,Waskom,Texas,US,75692


In [27]:
SQL_Statement = "SELECT ID, ADDRESS_CITY, ADDRESS_STATE, ADDRESS_COUNTRY, ADDRESS_POSTCODE \
                         FROM user_data "
UsersDF = pd.read_sql(SQL_Statement,connection)
UsersDF.to_csv('../Reports/Users.csv', index=False)

In [28]:
SQL_Statement = "SELECT ADDRESS_STATE, count(*) AS USER_COUNT \
                         FROM user_data \
                         GROUP BY ADDRESS_STATE \
                         ORDER BY USER_COUNT DESC"
UsersByStateDF = pd.read_sql(SQL_Statement,connection)
UsersByStateDF.to_csv('../Reports/UsersByState.csv', index=False)

In [30]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os.path

from config import from_email, email_password, to_email

email = from_email
password = email_password
send_to_email = to_email
subject = 'Sending User reports'
message = 'Please find the attachments with Users and Users by States, From Sushma Kesamsetty'
file1_location = '../Reports/Users.csv'
file2_location = '../Reports/UsersByState.csv'

msg = MIMEMultipart()
msg['From'] = from_email
msg['To'] = to_email
msg['Subject'] = subject

msg.attach(MIMEText(message, 'plain'))

# Setup the attachment
filename = os.path.basename(file1_location)
attachment = open(file1_location, "rb")
part = MIMEBase('application', 'octet-stream')
part.set_payload(attachment.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

# Attach the attachment to the MIMEMultipart object
msg.attach(part)

# Setup the attachment
filename = os.path.basename(file2_location)
attachment = open(file2_location, "rb")
part = MIMEBase('application', 'octet-stream')
part.set_payload(attachment.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

# Attach the attachment to the MIMEMultipart object
msg.attach(part)

server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.ehlo()
# server.starttls()
server.login(from_email, password)
text = msg.as_string()
server.sendmail(from_email, to_email, text)
server.quit()

(221, b'2.0.0 closing connection t1sm6887875pfj.115 - gsmtp')

In [29]:
session.close()